In [3]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch
import os
import linecache

In [6]:
# Data Preprocessing

CURRENT_PATH = os.getcwd()

def read_and_save(feature_path, files, label_path):

    for file in files:
        # Label

        filename = file.split('/')[-1] + '.pkl'
        
        # Data

        data = pd.read_csv(file,
                    sep = ' ', skiprows= 5 , header = None, comment='#')
        data.columns = pd.Index(['Index', 'Sensor', 'Position', 'Value'])
        data = data.drop(columns= ['Index'])
        pd.to_pickle(data, os.path.join(os.getcwd(), feature_path, filename))

        alcoholic = 1 if 'a' in filename else 0
        stimulus = ''.join(linecache.getline(file, 4).split()[1:3])
        with open(os.path.join(os.getcwd(), label_path), 'a') as file:
            file.write(stimulus + ' ' + str(alcoholic) + ' ' + filename+ '\n')

        
        
def process_directory(path, feature_path, label_path):
    print(f"Processing directory: {path}")

    subdirs = [os.path.join(path, name) for name in os.listdir(path)]
    if os.path.isfile(subdirs[0]):
        read_and_save(feature_path, subdirs, label_path)
    else:
        for dir_ in subdirs:
            process_directory(dir_, feature_path, label_path)


process_directory(os.path.join(CURRENT_PATH, 'data/EEG/SMNI_CMI_TEST/'), 'data/eeg/test/', 'data/eeg/test_label.txt')
process_directory(os.path.join(CURRENT_PATH, 'data/EEG/SMNI_CMI_TRAIN/'), 'data/eeg/train/', 'data/eeg/train_label.txt')

Processing directory: /home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/data/EEG/SMNI_CMI_TEST/
Processing directory: /home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/data/EEG/SMNI_CMI_TEST/co2a0000370
Processing directory: /home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/data/EEG/SMNI_CMI_TEST/co2a0000365
Processing directory: /home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/data/EEG/SMNI_CMI_TEST/co2a0000368
Processing directory: /home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/data/EEG/SMNI_CMI_TEST/co2c0000337
Processing directory: /home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/data/EEG/SMNI_CMI_TEST/co2c0000341
Processing directory: /home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/data/EEG/SMNI_CMI_TEST/co2a0000371
Processing directory: /home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/data/EEG/SMNI_CMI_TEST/co2c0000346
Processing directory: /home/hieunguyen/Desktop/MlSupervise/Experiment/experiment/da

In [3]:
# %load './data_scripts/eeg_data.py'



class EegDataset(Dataset):
    def __init__(self,tgt_transform = None, transform = None, train = True, channel = 'S1obj'):
        self.channel = channel
        self.train = train
        self.label_path = 'data/eeg/train_label.txt' if train else 'data/eeg/test_label.txt'
        self.labels = pd.read_csv(self.label_path, sep=' ', header= None)
        self.labels = self.labels[self.labels[0] == self.channel]
        self.feature_dir = 'data/eeg/train/' if train else 'data/eeg/test/'
        self.transform = transform
        self.tgt_transform = tgt_transform

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        
        feature_path = os.path.join(self.feature_dir, self.labels.iloc[index, 2])
        label = self.labels.iloc[index, 1]
        feature_matrix = pd.read_pickle(feature_path)
        if self.transform:
            feature_matrix = self.transform(feature_matrix)
        if self.tgt_transform:
            label = self.tgt_transform(label)
        
        return feature_matrix, label
        
        
def transform(x):
    assert(x.shape == (256 * 64, 3))
    assert(list(x.columns) == ['Sensor', 'Position', 'Value'])
    value = x['Value']
    value = torch.from_numpy(value.to_numpy()).reshape(256, 64)
    return value   

In [4]:


train_set = EegDataset(tgt_transform=lambda x: torch.tensor(x), transform=lambda x: transform(x))
train_loader = DataLoader(train_set, batch_size = 5, shuffle= True)

In [40]:
# model and Nesterov optimizer

class TraceLayer(nn.Module):
    def __init__(self, input_shape, binary = False):
        super(TraceLayer, self).__init__()
        self.input_shape = input_shape
        self.ai = nn.Parameter(torch.empty(input_shape))
        nn.init.xavier_uniform(self.ai)
        self.binary = binary
    
    def forward(self, x):
        linear = torch.sum(x * self.ai, dim = [1,2])
        
        if not self.binary:
            return linear
        return F.sigmoid(linear)
    
        
class Nesterov(torch.optim.Optimizer):
    def __init__(self, params, lr = 0.01, alpha_0 = 0, alpha_1 = 1):
        defaults = dict(lr = lr, alpha_0 = alpha_0, alpha_1 = alpha_1, old_params = params)
        super(Nesterov, self).__init__(params, defaults)
    
    def __setstate__(self, state):
        return super().__setstate__(state)
    
    def step(self, closure = None):

        if closure is None:
            raise ValueError("Closure required for recompute of the loss")
        
        for group in self.param_groups:

            for B_t in group['params']:

                S_t
        

    



In [41]:
model = TraceLayer((10,10))
model.forward(torch.randn((10,10,10)))

/tmp/ipykernel_22321/1541360345.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.ai)


tensor([-1.1034, -1.6551, -6.0736, -6.1800, -1.4062,  5.9677, -5.3846, -0.3019,
        -1.5379, -0.8733], grad_fn=<SumBackward1>)